In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
data_train = pd.read_csv('sources/train.csv')
data_test = pd.read_csv('sources/test2.csv')

In [3]:
X = data_train.drop(['Y'], axis='columns').values
print(X.shape)
X = np.expand_dims(X, axis=2)
print(X.shape)

y = data_train['Y'].values

X_train,X_dev,y_train,y_dev = train_test_split(X, y, test_size=0.2, \
                                               random_state=227, shuffle=True, stratify=y)

X_test = data_test.values


(1112, 6)
(1112, 6, 1)


In [4]:
def calculation_classes(y, y_test = None):
    print ('\t\tTRAIN DATA\tTEST DATA')
    print ('Negative\t', np.sum(y==0),'  ',
           round(100*np.sum(y==0)/y.shape[0],2),'%',
           '\t',np.sum(y_test==0),'  ',
           round(100*np.sum(y_test==0)/y_test.shape[0],2),'%'
           '\nPositive\t ', np.sum(y==1),' ',
           round(100*np.sum(y==1)/y.shape[0],2),'%'
           '\t',np.sum(y_test==1),' ',
           round(100*np.sum(y_test==1)/y_test.shape[0],2),'%'
      )

In [5]:
calculation_classes(y, y_dev)

		TRAIN DATA	TEST DATA
Negative	 1000    89.93 % 	 201    90.13 %
Positive	  112   10.07 %	 22   9.87 %


In [6]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
#from keras.preprocessing import sequence
#from keras.initializers import glorot_uniform
np.random.seed(1)
print('')

Using TensorFlow backend.


In [7]:
Tx=  6
Ty = 1

In [8]:
def model(input_shape, Ty):
    i = Input(shape=input_shape, dtype='float32')
    X = LSTM(32, return_sequences=False)(i)
    X = Dense(Ty, activation=None)(X)
    #X = Activation('softmax')(X)
    model = Model(inputs=[i], outputs=X)
    
    return model
    


In [9]:
LSTM_model = model((Tx,1),Ty)
LSTM_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6, 1)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                4352      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________


In [10]:
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
LSTM_model.fit(X_train, y_train , epochs = 50, batch_size = 32, shuffle=True,validation_data=(X_dev, y_dev))

Train on 889 samples, validate on 223 samples
Epoch 1/50
889/889 [==============================] - 1s 1ms/step - loss: 0.1929 - acc: 0.9123 - val_loss: 0.1560 - val_acc: 0.9372
Epoch 2/50
889/889 [==============================] - 0s 123us/step - loss: 0.1428 - acc: 0.9426 - val_loss: 0.1354 - val_acc: 0.9417
Epoch 3/50
889/889 [==============================] - 0s 139us/step - loss: 0.1265 - acc: 0.9438 - val_loss: 0.1303 - val_acc: 0.9507
Epoch 4/50
889/889 [==============================] - 0s 149us/step - loss: 0.1199 - acc: 0.9449 - val_loss: 0.1279 - val_acc: 0.9372
Epoch 5/50
889/889 [==============================] - 0s 152us/step - loss: 0.1137 - acc: 0.9449 - val_loss: 0.1225 - val_acc: 0.9417
Epoch 6/50
889/889 [==============================] - 0s 153us/step - loss: 0.1091 - acc: 0.9393 - val_loss: 0.1180 - val_acc: 0.9327
Epoch 7/50
889/889 [==============================] - 0s 163us/step - loss: 0.1042 - acc: 0.9314 - val_loss: 0.1145 - val_acc: 0.9103
Epoch 8/50
889/889